### קוד מבוא

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import py_scripts.functions
import py_scripts.statistical_areas_layer
from py_scripts.functions import drop_geo
from py_scripts.functions import make_point
from py_scripts.functions import up_load_df
from py_scripts.functions import up_load_gdb
from py_scripts.functions import up_load_shp
from py_scripts.statistical_areas_layer import statistical_areas_layer
from py_scripts.authority_level_information import authority_level_information
from py_scripts.upload_population_information import upload_population_information
from py_scripts.create_age_distribution_table import create_age_distribution_table
from py_scripts.percentage_of_hardi_of_statistical_area import percentage_of_hardi_of_statistical_area
from py_scripts.fix_level_a_arab_jerus import fix_level_a_arab_jerus
from py_scripts.assimilation_fix_level_a import assimilation_fix_level_a
from py_scripts.uploading_problematic_statistical_area_for_correction_level_b import uploading_problematic_statistical_area_for_correction_level_b
from py_scripts.create_GM_according_to_catalog import create_GM_according_to_catalog
from py_scripts.fix_lebel_b import fix_lebel_b
from py_scripts.creating_apartments_according_to_household_size import creating_apartments_according_to_household_size
from py_scripts.information_control_lms_at_authority_level import information_control_lms_at_authority_level
from py_scripts.export_stat_area_level_information import export_stat_area_level_information

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [4]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]


### העלת טבלת המרה בין למס מרכזי למשני

In [5]:
stat_join_from_main_to_secondary=up_load_df(
        r'{}\background_files'.format(software_data_folder_location),'stat_join_from_main_to_secondary')

### העלת טבלת יח_ד למ_ס

In [6]:
folder_path=r'{}\background_files'.format(software_data_folder_location)
file_name='מספר דירות לפי אזורים סטטיסטיים'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df)
df=df.dropna(how='all')


stat_aprt=df

In [7]:
col=['STAT', 'aprt_20']

In [8]:
stat_aprt=stat_aprt[col]

### שכבת אזורים סטטיסטים

In [9]:
path=r'{}\background_files\statisticalareas_2020_demography.gdb'.format(software_data_folder_location)
layer_list=fiona.listlayers(path)

layer_name='statisticalareas_2020_demography_jtmt_area'

gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name)).fillna(0)

stat=gpd_layer.rename(columns={'YISHUV_STAT11':'STAT'})

In [10]:
stat=statistical_areas_layer(stat, stat_aprt, stat_join_from_main_to_secondary)

###  מידע ברמת רשות

In [11]:
stat_point=make_point(stat[['STAT', 'geometry']])
muni_under_JTMT_ITM=up_load_gdb(r'{}\background_files\MUNI_border.gdb'.format(software_data_folder_location),'muni_under_JTMT_ITM')
muni_under_JTMT_ITM=muni_under_JTMT_ITM[['CR_PNIM','geometry']]

In [12]:
stat=authority_level_information(stat, stat_point, muni_under_JTMT_ITM)

###  העלת מידע אוכלוסייה

In [13]:
folder_path=r'{}\background_files'.format(software_data_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='classification')
df=df.dropna(how='all')

classification=df

In [14]:
stat=upload_population_information(stat, classification)

In [15]:
folder_path=r'{}\background_files'.format(software_data_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='hh_size')
df=df.dropna(how='all')

hh_size=df

In [16]:
stat=pd.merge(stat,hh_size,on='classification_name',how='left')

In [17]:
folder_path=r'{}\background_files'.format(software_data_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='types')
df=df.dropna(how='all')

age_des_types=df

In [18]:
folder_path=r'{}\background_files'.format(software_data_folder_location)
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='absolute_numbers')
df=df.dropna(how='all').fillna(0)

cbs_pop=df

In [19]:
cbs_pop['pop']=cbs_pop.iloc[:,3:].sum(axis=1)

In [20]:
age=['0','5','10','15','20','25','30','35','40','45','50','55','60','65','70','75','80','85']

In [21]:
for x in age:
    cbs_pop['pop_{}'.format(x)]= cbs_pop['female_{}'.format(x)]+cbs_pop['male_{}'.format(x)]

In [22]:
cbs_pop['pop_75up']= cbs_pop[['pop_75', 'pop_80', 'pop_85']].sum(axis=1)

In [23]:
col=[ 'main_stat', 'pop', 'pop_0', 'pop_5', 'pop_10', 'pop_15', 'pop_20', 'pop_25', 'pop_30', 'pop_35', 'pop_40', 'pop_45', 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [24]:
cbs_pop=cbs_pop[col]

יצירת טבלת התפלגות גילים באחוזים לטובת המשך הקוד כאשר יש שינויים בסך הכל אוכלוסיה אבל מעוניינים עדיין בהתפלגות גילים המקורית

In [25]:
cbs_pop_pre=cbs_pop.copy()

In [26]:
stat=create_age_distribution_table(cbs_pop_pre, cbs_pop, stat)

###  אחוז חרדים מכלל א"ס

In [27]:
folder_path=r'{}\background_files'.format(software_data_folder_location)
file_name='אוכלוסייה חרדית לפי מחוז, יישוב, ואזור סטטיסטי - 2020'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df)
df=df.dropna(how='all')

stat_hardi=df

In [28]:
stat=percentage_of_hardi_of_statistical_area(stat_hardi, stat)

###  תיקון שלב א ערבי ירושלים

In [29]:
path=r'{}\background_files\statisticalareas_2020_demography.gdb'.format(software_data_folder_location)
layer_name='statisticalareas_2020_demography_arab_jtmt'
stat_arab=up_load_gdb(path,layer_name)

In [30]:
stat_arab=fix_level_a_arab_jerus(col, stat_arab, cbs_pop_pre)

c:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\py_scripts\fix_level_a_arab_jerus.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-103981.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_pop_jtmt.loc['east_jeru_left','pop']=group_pop_delta.item()


###  הטמעת תיקון שלב א

In [31]:
stat=assimilation_fix_level_a(stat_arab, stat)

###  העלת א"ס בעייתי לתיקון שלב ב

In [32]:
path=r'{}\background_files\jtmt_fix_for_cbs_data_2020.shp'.format(software_data_folder_location)

jtmt_fix_stat=up_load_shp(path)

In [33]:
stat=uploading_problematic_statistical_area_for_correction_level_b(jtmt_fix_stat, stat)

###  יצירת גמ לפי קטלוג

In [35]:
stat=create_GM_according_to_catalog(stat, hh_size)

c:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\py_scripts\create_GM_according_to_catalog.py:12: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  hh_size_by_classification=drop_geo(stat).loc[(stat['aprt_20']>0)&(stat['STAT'].isin(cbs_not_lie_lst))].pivot_table(index='classification_name',aggfunc=sum)[['aprt_20','pop','count']]


###  תיקון שלב ב

In [36]:
stat=fix_lebel_b(stat)

###  יצירת דירות לפי גודל משק בית

In [37]:
stat=creating_apartments_according_to_household_size(stat)

### בקרת מידע למס ברמה של רשות


In [38]:
pop_2020_cbs_muni=up_load_df(r'{}\background_files'.format(software_data_folder_location),'pop_2020_cbs_muni')

In [39]:
stat=information_control_lms_at_authority_level(stat, pop_2020_cbs_muni)

c:\Users\dpere\Documents\JTMT\forecast\create_forecast_basic\current\py_scripts\information_control_lms_at_authority_level.py:6: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  stat_by_muni_sum=drop_geo(stat).pivot_table(index='CR_PNIM',aggfunc=sum)[['pop_cbs','pop']]


### ייצא מידע ברמת אזור סטט עם השינוים שלנו

In [40]:
stat=export_stat_area_level_information(stat)

In [41]:
col=['STAT','precent_of_stat_data','classification_name','pop','pop_cbs','aprt_20','aprt_20_cbs','pop_delta','aprt_20_delta','change_from_cbs']

In [42]:
stat[col].query('STAT!=5526 & STAT!=9975').to_excel(r'{}\Intermediates\stat_cbs_jtmt_2020_short.xlsx'.format(software_data_folder_location),index=False)

In [43]:
stat.query('STAT!=5526 & STAT!=9975').to_excel(r'{}\Intermediates\stat_cbs_jtmt_2020.xlsx'.format(software_data_folder_location),index=False)